In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import pickle
import matplotlib.dates as mdates
from datetime import datetime

In [2]:
from sqlalchemy import create_engine
USER = "ciara"
PASSWORD = "Ciara123"
PORT = "3306"
DB = "dbbikes"
URI = "dbbikes.cvr6gofoxmkp.us-east-2.rds.amazonaws.com"
engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [3]:
sql = f"""
SELECT id, number, updated_on, available_bikes, available_bike_stands
FROM dbbikes_info
WHERE updated_on is not null and updated_on >= '2021-03-24 19:25:16' and number != 507
GROUP BY id, day(updated_on)
ORDER by id desc, updated_on desc;
"""

In [4]:
bike_df = pd.read_sql_query(sql, engine)

2021-04-14 22:52:13,698 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-04-14 22:52:13,699 INFO sqlalchemy.engine.base.Engine {}
2021-04-14 22:52:13,850 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-04-14 22:52:13,851 INFO sqlalchemy.engine.base.Engine {}
2021-04-14 22:52:14,103 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2021-04-14 22:52:14,104 INFO sqlalchemy.engine.base.Engine {}
2021-04-14 22:52:14,350 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8mb4' and `Collation` = 'utf8mb4_bin'
2021-04-14 22:52:14,351 INFO sqlalchemy.engine.base.Engine {}
2021-04-14 22:52:14,493 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2021-04-14 22:52:14,494 INFO sqlalchemy.engine.base.Engine {}
2021-04-14 22:52:14,618 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2021-04-14 22:52:14,619 INFO sqlalchemy.engine.base.E

In [5]:
bike_df['updated_on'] = bike_df['updated_on'].astype('datetime64[ns]')
bike_df['updated_on'] = pd.to_datetime(bike_df['updated_on'])
bike_df

,id,number,updated_on,available_bikes,available_bike_stands
0,1267838,88,2021-04-11 16:55:56,22,8
1,1267837,21,2021-04-11 16:52:37,9,21
2,1267836,92,2021-04-11 16:58:58,6,34
3,1267835,83,2021-04-11 16:57:08,12,27
4,1267834,39,2021-04-11 16:52:23,3,17
...,...,...,...,...,...
552665,714514,52,2021-03-24 19:27:10,0,32
552666,714513,32,2021-03-24 19:28:35,10,20
552667,714511,6,2021-03-24 19:28:23,2,18
552668,714507,30,2021-03-24 19:29:17,7,13


In [6]:
bike_df.dtypes

id                                int64
number                            int64
updated_on               datetime64[ns]
available_bikes                   int64
available_bike_stands             int64
dtype: object

In [7]:
bike_df.shape

(552670, 5)

In [8]:
bike_df.tail()

,id,number,updated_on,available_bikes,available_bike_stands
552665,714514,52,2021-03-24 19:27:10,0,32
552666,714513,32,2021-03-24 19:28:35,10,20
552667,714511,6,2021-03-24 19:28:23,2,18
552668,714507,30,2021-03-24 19:29:17,7,13
552669,714506,42,2021-03-24 19:28:58,24,6


In [9]:
sql = f"""
SELECT * FROM weather;
"""

In [10]:
weather_df = pd.read_sql_query(sql, engine)

2021-04-14 22:52:27,916 INFO sqlalchemy.engine.base.OptionEngine 
SELECT * FROM weather;

2021-04-14 22:52:27,918 INFO sqlalchemy.engine.base.OptionEngine {}


In [11]:
weather_df.dtypes

id                      int64
description            object
temp                  float64
last_update    datetime64[ns]
dtype: object

In [12]:
# category    
weather_df['description'] = weather_df['description'].astype('category')    

# datetime   
weather_df['last_update'] = weather_df['last_update'].astype('datetime64[ns]')

In [13]:
weather_df.dtypes

id                      int64
description          category
temp                  float64
last_update    datetime64[ns]
dtype: object

In [14]:
weather_df.shape

(5921, 4)

In [15]:
weather_df.tail()

,id,description,temp,last_update
5916,5917,few clouds,4.21,2021-04-14 21:23:36
5917,5918,few clouds,4.14,2021-04-14 21:30:47
5918,5919,few clouds,4.14,2021-04-14 21:30:47
5919,5920,few clouds,3.66,2021-04-14 21:45:02
5920,5921,few clouds,3.66,2021-04-14 21:46:48


In [16]:
bike_df.sort_values('updated_on', inplace=True)
weather_df.sort_values('last_update', inplace=True) 

In [17]:
df = pd.merge_asof(bike_df, weather_df, left_on='updated_on', right_on='last_update')

In [18]:
df.shape

(552670, 9)

In [19]:
df = df.drop(['last_update'], axis=1)

In [20]:
df.tail(5)

,id_x,number,updated_on,available_bikes,available_bike_stands,id_y,description,temp
552665,1267774,7,2021-04-11 16:59:24,5,24,5000,moderate rain,7.0
552666,1267751,90,2021-04-11 16:59:29,16,23,5000,moderate rain,7.0
552667,1267803,22,2021-04-11 16:59:32,8,12,5000,moderate rain,7.0
552668,1267793,69,2021-04-11 16:59:36,16,23,5000,moderate rain,7.0
552669,1267831,29,2021-04-11 16:59:39,19,9,5000,moderate rain,7.0


In [21]:
df = df.drop(['id_y'], axis=1)

In [22]:
df = df.drop(['id_x'], axis=1)

In [23]:
df.tail(5)

,number,updated_on,available_bikes,available_bike_stands,description,temp
552665,7,2021-04-11 16:59:24,5,24,moderate rain,7.0
552666,90,2021-04-11 16:59:29,16,23,moderate rain,7.0
552667,22,2021-04-11 16:59:32,8,12,moderate rain,7.0
552668,69,2021-04-11 16:59:36,16,23,moderate rain,7.0
552669,29,2021-04-11 16:59:39,19,9,moderate rain,7.0


In [24]:
df.shape

(552670, 6)

In [25]:
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())
print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  252528
Number of duplicate rows (including first) in the table is: 496898


In [26]:
df = df.drop_duplicates(keep='last')
print('Duplicate remaining rows:', df[df.duplicated(keep='last')].shape[0])

Duplicate remaining rows: 0


In [27]:
df.shape

(300142, 6)

In [28]:
df

,number,updated_on,available_bikes,available_bike_stands,description,temp
1,28,2021-03-24 19:25:34,3,27,clear sky,7.01
3,29,2021-03-24 19:25:40,1,28,clear sky,7.01
5,96,2021-03-24 19:25:43,15,15,clear sky,7.01
6,34,2021-03-24 19:25:45,12,17,clear sky,7.01
8,38,2021-03-24 19:26:04,16,24,clear sky,7.01
...,...,...,...,...,...,...
552665,7,2021-04-11 16:59:24,5,24,moderate rain,7.00
552666,90,2021-04-11 16:59:29,16,23,moderate rain,7.00
552667,22,2021-04-11 16:59:32,8,12,moderate rain,7.00
552668,69,2021-04-11 16:59:36,16,23,moderate rain,7.00


In [29]:
df.shape

(300142, 6)

In [30]:
df['hourly'] = df['updated_on'].dt.hour
hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')
df = pd.concat([df, hourly_dummies], axis=1)
hour_dummies = list(hourly_dummies)
df = df.drop(['hourly'], axis=1)

df['weekday'] = df['updated_on'].dt.dayofweek

In [31]:
print(df.head(10))
print(df.keys())

    number          updated_on  available_bikes  available_bike_stands  \
1       28 2021-03-24 19:25:34                3                     27   
3       29 2021-03-24 19:25:40                1                     28   
5       96 2021-03-24 19:25:43               15                     15   
6       34 2021-03-24 19:25:45               12                     17   
8       38 2021-03-24 19:26:04               16                     24   
10     115 2021-03-24 19:26:14               20                     10   
12     105 2021-03-24 19:26:16                5                     31   
14      26 2021-03-24 19:26:16                5                     15   
16      84 2021-03-24 19:26:19                4                     26   
19       2 2021-03-24 19:26:23                3                     17   

   description  temp  hour_0  hour_1  hour_2  hour_3  ...  hour_15  hour_16  \
1    clear sky  7.01       0       0       0       0  ...        0        0   
3    clear sky  7.01       

In [32]:
stations = list(sorted(df.number.unique()))
weekdays = list(sorted(df.weekday.unique()))
models = {}

i = 0

for station in stations:
    models[station] = {}
    for weekday in weekdays:
        df1 = df[(df['number'] == station) & (df['weekday'] == weekday)]
        y = pd.DataFrame(df1, columns=['available_bikes', 'temp'])
        X = pd.DataFrame(df1, columns=['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        models[station][weekday] = LinearRegression().fit(X_train, y_train)
        model = LinearRegression().fit(X_train, y_train)
        
        train_predictions = model.predict(X_train)
        # print('Train predictions for station ', station, ": ", train_predictions)

        test_predictions = model.predict(X_test)
        # print('Test predictions for station ', station, ": ", test_predictions)

        
        r2 = metrics.r2_score(y_test, test_predictions)

        # how well does the model behave compared to a chosen r2 ? 
        if r2 > 0.5:             
            i +=1

print('Percentage of models that have outperformed r2 = 0.5:', (i /109) * 100)  # there are 109 stations
# print(i)
# print(len(stations))

# with open('model.pkl', 'wb') as handle:
#     pickle.dump(models, handle, protocol = pickle.HIGHEST_PROTOCOL)

Percentage of models that have outperformed r2 = 0.5: 44.95412844036697
